In [1]:
# 1 Paso - Instalar PySpark en el Contexto de Colab (si es que no está instalado)
!pip install -q PySpark

In [3]:
# 2 Paso - Conectarme y crear la sesión en Apache Spark (SparkSession)
from pyspark.sql import SparkSession
# Crear la sesión
spark = SparkSession.builder.appName("ProyectoMetroSantiago").config("spark.executor.memory","1g").config("spark.driver.memory","1g").config("spark.sql.shuffle.partitions","2").getOrCreate()

In [5]:
# Cargar el CSV en un DF (asegurarse de subir antes el archivo a la carpeta interna de recursos)
df = spark.read.csv("metro_santiago.csv",header=True,inferSchema=True)

#Explrar este DF (Apache Spark) con show
df.show()

+---------+-----+--------------+--------------------+----------------+-------------------+
|id_evento|línea|      estación|      tipo_incidente|duración_minutos|               hora|
+---------+-----+--------------+--------------------+----------------+-------------------+
|        1|   L2|      Lo Prado|     Falla eléctrica|              15|2025-08-23 11:45:00|
|        2|   L5|   Irarrázaval|          Evacuación|              15|2025-08-23 20:00:00|
|        3|   L4|     San Pablo|     Falla eléctrica|              25|2025-08-23 19:00:00|
|        4|   L1|      Lo Prado|     Falla eléctrica|              30|2025-08-23 05:45:00|
|        5|   L3|   Irarrázaval|     Falla eléctrica|              20|2025-08-23 08:00:00|
|        6|   L2|    Los Leones|       Mantenimiento|              30|2025-08-23 17:00:00|
|        7|   L3|         Ñuñoa|       Mantenimiento|               5|2025-08-23 12:30:00|
|        8|   L1|    Los Héroes|     Falla eléctrica|              30|2025-08-23 08:30:00|

In [7]:
# Transformar el df a RDD
rdd = df.rdd

# Mostrar algunos registros del RDD
rdd.take(5)

[Row(id_evento=1, línea='L2', estación='Lo Prado', tipo_incidente='Falla eléctrica', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 11, 45)),
 Row(id_evento=2, línea='L5', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 20, 0)),
 Row(id_evento=3, línea='L4', estación='San Pablo', tipo_incidente='Falla eléctrica', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 19, 0)),
 Row(id_evento=4, línea='L1', estación='Lo Prado', tipo_incidente='Falla eléctrica', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 5, 45)),
 Row(id_evento=5, línea='L3', estación='Irarrázaval', tipo_incidente='Falla eléctrica', duración_minutos=20, hora=datetime.datetime(2025, 8, 23, 8, 0))]

In [8]:
# Almacenar "in .memory" (en memoria cache) nuestro RDD para utilizarlo varias veces
rdd.cache()

MapPartitionsRDD[28] at javaToPython at NativeMethodAccessorImpl.java:0

In [9]:
# Ver cuantas particiones tiene nuestro RDD
rdd.getNumPartitions()

1

In [10]:
# Reparticionar nuestro RDD en 4 particiones
rdd_repartido = rdd.repartition(4)

rdd_repartido.getNumPartitions()

4

In [11]:
# Transformaciones - Son operaciones a realizar en el RDD que lo afectan, cerando un nuevo objeto RDD
rdd_filtrado = rdd.filter(lambda row: int(row['duración_minutos']) > 20)

rdd_filtrado.take(5)

[Row(id_evento=3, línea='L4', estación='San Pablo', tipo_incidente='Falla eléctrica', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 19, 0)),
 Row(id_evento=4, línea='L1', estación='Lo Prado', tipo_incidente='Falla eléctrica', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 5, 45)),
 Row(id_evento=6, línea='L2', estación='Los Leones', tipo_incidente='Mantenimiento', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 17, 0)),
 Row(id_evento=8, línea='L1', estación='Los Héroes', tipo_incidente='Falla eléctrica', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 8, 30)),
 Row(id_evento=9, línea='L3', estación='Tobalaba', tipo_incidente='Evacuación', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 16, 15))]

In [12]:
# TRANSFORMACIÓN - FILTER: aplica un filtro o modifica el RDD en base a un criterio lógico
rdd_evacuaciones = rdd.filter(lambda row: row['tipo_incidente'] == 'Evacuación')
rdd_evacuaciones.take(5)

[Row(id_evento=2, línea='L5', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 20, 0)),
 Row(id_evento=9, línea='L3', estación='Tobalaba', tipo_incidente='Evacuación', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 16, 15)),
 Row(id_evento=11, línea='L2', estación='Plaza Egaña', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 12, 15)),
 Row(id_evento=15, línea='L4', estación='Los Héroes', tipo_incidente='Evacuación', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 15, 15)),
 Row(id_evento=18, línea='L3', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 5, 30))]

In [13]:
# TRANSFORMACIÓN - MAP: Función que aplica a cada elemento del RDD y devuelve uno nuevo transformado
rdd_estaciones_mayusculas = rdd.map(lambda row: row['estación'].upper())
rdd_estaciones_mayusculas.take(5)

['LO PRADO', 'IRARRÁZAVAL', 'SAN PABLO', 'LO PRADO', 'IRARRÁZAVAL']

In [15]:
# TRANSFORMACIÓN - FLATMAP: Función que, como map(),permite devolver elementos transformados pero varios de una sola entrada
rdd_estaciones = rdd.map(lambda row: row['estación'])
palabras = rdd_estaciones.flatMap(lambda x: x.split(" "))
palabras.take(5)

['Lo', 'Prado', 'Irarrázaval', 'San', 'Pablo']

In [20]:
# TRANSFORMACIÓN - SAMPLE: Permite extraer una muestra aleatoria del RDD
rdd_muestra = rdd.sample(seed=42,fraction=0.3,withReplacement=False)
rdd_muestra.take(5)

[Row(id_evento=10, línea='L4A', estación='Los Dominicos', tipo_incidente='Interrupción de servicio', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 16, 45)),
 Row(id_evento=11, línea='L2', estación='Plaza Egaña', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 12, 15)),
 Row(id_evento=14, línea='L5', estación='Pajaritos', tipo_incidente='Mantenimiento', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 13, 30)),
 Row(id_evento=17, línea='L1', estación='Vicente Valdés', tipo_incidente='Falla eléctrica', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 20, 15)),
 Row(id_evento=18, línea='L3', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 5, 30))]

In [21]:
# TRANSFORMACIÓN - UNION: Une 2 RDDs (sin eliminar duplicados)
rdd_duplicado = rdd.filter(lambda row: row['tipo_incidente'] == "Falla eléctrica")
rdd_unido = rdd_evacuaciones.union(rdd_duplicado)
rdd_unido.take(5)

[Row(id_evento=2, línea='L5', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 20, 0)),
 Row(id_evento=9, línea='L3', estación='Tobalaba', tipo_incidente='Evacuación', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 16, 15)),
 Row(id_evento=11, línea='L2', estación='Plaza Egaña', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 12, 15)),
 Row(id_evento=15, línea='L4', estación='Los Héroes', tipo_incidente='Evacuación', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 15, 15)),
 Row(id_evento=18, línea='L3', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 5, 30))]

In [22]:
# TRANSFORMACIÓN - DISTINCT: Elimina los duplicados (igual que SQL)
rdd_estaciones_distintas = rdd.map(lambda row: row['estación']).distinct()
rdd_estaciones_distintas.take(5)

['Lo Prado', 'Irarrázaval', 'San Pablo', 'Los Leones', 'Ñuñoa']

In [23]:
# TRANSFORMACIÓN - SORTBY: Ordena los elementos segun criterio
rdd_ordenado = rdd.sortBy(lambda row: int(row['duración_minutos']))
rdd_ordenado.take(5)

[Row(id_evento=7, línea='L3', estación='Ñuñoa', tipo_incidente='Mantenimiento', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 12, 30)),
 Row(id_evento=13, línea='L5', estación='Los Héroes', tipo_incidente='Falla eléctrica', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 17, 0)),
 Row(id_evento=24, línea='L5', estación='Pajaritos', tipo_incidente='Mantenimiento', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 9, 15)),
 Row(id_evento=25, línea='L2', estación='Tobalaba', tipo_incidente='Falla eléctrica', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 23, 15)),
 Row(id_evento=40, línea='L5', estación='Los Dominicos', tipo_incidente='Evacuación', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 18, 45))]